In [6]:

%matplotlib inline

from pathlib import Path
from datetime import date
import json
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
import requests
import numpy as np
import pandas as pd

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import multiprocessing

In [7]:
sns.set_style('whitegrid')
# store data in this directory since we won't use it in other chapters
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data

In [8]:
data_path

WindowsPath('c:/Users/fuway/Desktop/stocks')

In [9]:
symbols = pd.read_csv(data_path/"stock price data"/"finer_data_complete_stock.csv").dropna()
symbols

,Symbol,Sector,Industry
0,A,Healthcare,Diagnostics & Research
1,AAL,Industrials,Airlines
2,AAN,Industrials,Rental & Leasing Services
3,AAON,Industrials,Building Products & Equipment
4,AAP,Consumer Cyclical,Specialty Retail
...,...,...,...
2112,ZEUS,Basic Materials,Steel
2113,ZION,Financial Services,Banks—Regional
2114,ZIONP,Financial Services,Banks—Regional
2115,ZUMZ,Consumer Cyclical,Apparel Retail


In [5]:
data = pd.read_csv(data_path/"fundamental data"/"company financials"/"ADES"/"ADES_fundamentals_Quarter.csv")
data.columns

Index(['Quarters', 'Year', 'Quarter', 'FillingDate', 'Date', 'NetIncome',
       'Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit',
       'GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
       'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',
       'MarketCap', 'EnterpriseValue', 'bookValuePerShare', 'currentRatio',
       'debtToAssets', 'debtToEquity', 'roic', 'shareholdersEquityPerShare',
       'roe', 'priceToSalesRatio', 'peRatio', 'pbRatio', 'payoutRatio',
       'inventoryTurnover', 'dividendYield', 'assetTurnover', 'quickRatio',
       'returnOnAssets', 'priceToFreeCashFlowsRatio', 'grossProfitMargin',
       'epsdilutedGrowth', 'fiveYNetIncomeGrowthPerShare',
       'fiveYRevenueGrowthPerShare', 'fiveYShareholdersEquityGrowthPerShare',
       'grossProfitGrowth', 'netIncomeGrowth', 'operatingIncomeGrowth',
       'threeYNetIncomeGrowthPerShare', 'threeYRevenueGrowthPerShare',
       'threeYShareholdersEquityGrowthPerShare

In [6]:
data[['Quarters', 'Year', 'Quarter', 'NetIncome',
       'Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit',
       'GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
       'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',
       'MarketCap', 'EnterpriseValue',]].interpolate(method='linear').dropna().loc[33:39,:]

,Quarters,Year,Quarter,NetIncome,Revenue,OperatingIncome,EPSDiluted,GrossProfit,GrossProfitRatio,TotalAssets,CommonStockEquity,CurrentAssets,CurrentLiability,TotalLiability,Inventory,RetainedEarnings,MarketCap,EnterpriseValue
33,2014Q4,2014,Q4,4677000.0,3693000.0,-15546000.0,0.22000,790000.0,0.213918,93699000.0,-697000.0,54429000.0,47157000.0,94396000.0,630000.0,-110888000.0,494451840.0,4.851808e+08
34,2014Q3,2014,Q3,-3699000.0,9072000.0,-10279000.0,-0.17000,2560000.0,0.282187,88655250.0,-2064500.0,54660500.0,46007500.0,90719750.0,505000.0,-111234750.0,458070720.0,4.999190e+08
35,2014Q2,2014,Q2,992000.0,3174000.0,-8420000.0,0.04000,1421000.0,0.447700,83611500.0,-3432000.0,54892000.0,44858000.0,87043500.0,380000.0,-111581500.0,492467610.0,5.146572e+08
36,2014Q1,2014,Q1,-583000.0,983000.0,-7570000.0,-0.03000,532000.0,0.541200,78567750.0,-4799500.0,55123500.0,43708500.0,83367250.0,255000.0,-111928250.0,526751100.0,5.293953e+08
37,2013Q4,2013,Q4,-12231000.0,-188551000.0,-40894000.0,-0.56981,-39497000.0,-0.209476,73524000.0,-6167000.0,55355000.0,42559000.0,79691000.0,130000.0,-112275000.0,582023475.0,5.441335e+08
38,2013Q3,2013,Q3,1591000.0,74593000.0,7769000.0,0.08000,19133000.0,0.256499,106415000.0,-18032000.0,56450000.0,71719000.0,152512000.0,0.0,-83521000.0,431899200.0,4.196392e+08
39,2013Q2,2013,Q2,-3178000.0,58930000.0,561000.0,-0.16000,10594000.0,0.179773,86987000.0,-20318000.0,36942000.0,54927000.0,129269000.0,0.0,-85112000.0,413116000.0,4.034140e+08


In [5]:
[name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]

['A',
 'AA',
 'AAC',
 'AACG',
 'AACI',
 'AACIW',
 'AADI',
 'AAIC',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHW',
 'ACAQ',
 'ACAX',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACDCW',
 'ACEL',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACP',
 'ACR',
 'ACRE',
 'ACRO',
 'ACRS',
 'ACRV',
 'ACRX',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACV',
 'ACVA',
 'ACXP',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADEA',
 'ADER',
 'ADERW',
 'ADES',
 'ADEX',
 'ADI',
 'ADIL',
 'ADILW',
 'ADM',
 'ADMA',
 'ADMP',
 'ADN',
 'AD

In [18]:

set(b)

{'AssetsTurnover',
 'AvgVolume',
 'CurrentRatio',
 'CurrentRatioIncrease',
 'Date',
 'DebtToEquity',
 'DebtToEquityIncrease',
 'EnterpriseValue',
 'FillingDate',
 'GrossProfitGrowth',
 'Industry',
 'MarketCap',
 'NetIncomeGrowth',
 'NetProfitMargin',
 'NetProfitMarginIncrease',
 'OperatingIncomeGrowth',
 'OperatingMargin',
 'OperatingMarginIncrease',
 'Price',
 'ReturnOnAssets',
 'ReturnOnAssetsIncrease',
 'ReturnOnEquity',
 'ReturnOnEquityIncrease',
 'RevenueGrowth',
 'Sector',
 'Symbol',
 'Year',
 'bookValuePerShare',
 'debtToAssets',
 'dividendYield',
 'epsdilutedGrowth',
 'grossProfitMargin',
 'inventoryTurnover',
 'payoutRatio',
 'pbRatio',
 'peRatio',
 'priceToFreeCashFlowsRatio',
 'priceToSalesRatio',
 'quickRatio',
 'quickRatioIncrease',
 'roic',
 'roicIncrease'}

In [16]:
a = ['NetIncome',
       'Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit',
       'GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
       'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',"assetTurnover","debtToEquity","currentRatio","roe","returnOnAssets",'shareholdersEquityPerShare','threeYNetIncomeGrowthPerShare', 'threeYRevenueGrowthPerShare',
       'threeYShareholdersEquityGrowthPerShare','fiveYNetIncomeGrowthPerShare', 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare',"netIncomeGrowth","grossProfitGrowth","operatingIncomeGrowth"] 
b = year_file.columns
set(a).issubset(set(b))

False

In [19]:
ttm_data = pd.DataFrame()

# for symbol in symbols.Symbol:
for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
    path = data_path/"fundamental data"/"company financials"/symbol
    file_year = symbol+"_fundamentals_Year.csv"
    file_quarter = symbol+"_fundamentals_Quarter.csv"
    year_file = pd.read_csv(path/file_year)
    quarter_file = pd.read_csv(path/file_quarter)
    #do not fill TTM, interpolation fill followed by forward fill and backward fill
    ttm = year_file[year_file["Year"]=="TTM"]
    ttm_data = pd.concat([ttm_data, ttm], ignore_index=True)
    year_file = year_file[year_file["Year"]!="TTM"].sort_values(by="Year")
    year_file = year_file.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
    columns = ['NetIncome','Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit','GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
       'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',"assetTurnover","debtToEquity","currentRatio","roe","returnOnAssets",'shareholdersEquityPerShare','threeYNetIncomeGrowthPerShare', 'threeYRevenueGrowthPerShare',
       'threeYShareholdersEquityGrowthPerShare','fiveYNetIncomeGrowthPerShare', 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare',"netIncomeGrowth","grossProfitGrowth","operatingIncomeGrowth"]
    
    if set(columns).issubset(set(year_file.columns)):
        year_file["ReturnOnAssets"] = year_file.apply(lambda row:row["NetIncome"]/row["TotalAssets"] if row["TotalAssets"] >0 else None, axis=1)
        year_file["ReturnOnAssets"].fillna(year_file["returnOnAssets"], inplace=True)
        year_file["ReturnOnEquity"] = year_file.apply(lambda row:row["NetIncome"]/row["CommonStockEquity"] if row["CommonStockEquity"] >0 else None, axis=1)
        year_file["ReturnOnEquity"].fillna(year_file["roe"], inplace=True)
        year_file["CurrentRatio"] = year_file.apply(lambda row:row["CurrentAssets"]/row["CurrentLiability"] if row["CurrentLiability"] >0 else None, axis=1)
        year_file["CurrentRatio"].fillna(year_file["currentRatio"], inplace=True)
        year_file["OperatingMargin"] = year_file.apply(lambda row:row["OperatingIncome"]/row["Revenue"] if row["Revenue"] >0 else None, axis=1)
        year_file["NetProfitMargin"] = year_file.apply(lambda row:row["NetIncome"]/row["Revenue"] if row["Revenue"] >0 else None, axis=1)
        year_file["DebtToEquity"] = year_file.apply(lambda row:row["TotalLiability"]/row["CommonStockEquity"] if row["CommonStockEquity"] >0 else None, axis=1)
        year_file["DebtToEquity"].fillna(year_file["debtToEquity"], inplace=True)
        year_file["AssetsTurnover"] = year_file.apply(lambda row:row["Revenue"]/row["TotalAssets"] if row["TotalAssets"] >0 else None, axis=1)
        year_file["AssetsTurnover"].fillna(year_file["assetTurnover"], inplace=True)
        year_file.fillna(0,inplace=True)
        for i in ['roic','quickRatio','ReturnOnAssets', 'ReturnOnEquity', 'CurrentRatio', 'OperatingMargin','NetProfitMargin', 'DebtToEquity',]:
            year_file[i+"Increase"] = year_file[i].diff().fillna(method="bfill")
        for i in ['NetIncome', 'Revenue', 'OperatingIncome', 'GrossProfit']:
            year_file[i+"Growth"] = year_file[i].shift()
            year_file[i+"Growth"] = year_file.apply(lambda row:(row[i] - row[i+"Growth"])/row[i+"Growth"] if row[i+"Growth"] >0 else None, axis=1)
        year_file["NetIncomeGrowth"].fillna(year_file["netIncomeGrowth"], inplace=True)
        year_file["OperatingIncomeGrowth"].fillna(year_file["operatingIncomeGrowth"], inplace=True)
        year_file["GrossProfitGrowth"].fillna(year_file["grossProfitGrowth"], inplace=True)
        year_file_numeric = year_file.select_dtypes(include=[np.number])
        year_file[year_file_numeric.columns] = year_file_numeric.fillna(year_file_numeric.mean())
        year_file.drop(['NetIncome',
        'Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit',
        'GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
        'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',"assetTurnover","debtToEquity","currentRatio","roe","returnOnAssets",'shareholdersEquityPerShare','threeYNetIncomeGrowthPerShare', 'threeYRevenueGrowthPerShare',
        'threeYShareholdersEquityGrowthPerShare','fiveYNetIncomeGrowthPerShare', 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare',"netIncomeGrowth","grossProfitGrowth","operatingIncomeGrowth"], axis=1, inplace=True)
        year_file_numeric = year_file.select_dtypes(include=[np.number])
        year_file[year_file_numeric.columns] = year_file_numeric.fillna(year_file_numeric.mean())

        new_path = data_path/"fundamental data"/"company ratios"/symbol
        if not new_path.exists():
            new_path.mkdir(parents=True)
        file_year = symbol+"_ratios_Year.csv"
        year_file.to_csv(new_path/file_year,index=False)
    if set(columns).issubset(set(quarter_file.columns)):
        quarter_file = quarter_file.sort_values(by="Quarters")
        quarter_file = quarter_file.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

        quarter_file["ReturnOnAssets"] = quarter_file.apply(lambda row:row["NetIncome"]/row["TotalAssets"] if row["TotalAssets"] >0 else None, axis=1)
        quarter_file["ReturnOnAssets"].fillna(quarter_file["returnOnAssets"], inplace=True)
        quarter_file["ReturnOnEquity"] = quarter_file.apply(lambda row:row["NetIncome"]/row["CommonStockEquity"] if row["CommonStockEquity"] >0 else None, axis=1)
        quarter_file["ReturnOnEquity"].fillna(quarter_file["roe"], inplace=True)
        quarter_file["CurrentRatio"] = quarter_file.apply(lambda row:row["CurrentAssets"]/row["CurrentLiability"] if row["CurrentLiability"] >0 else None, axis=1)
        quarter_file["CurrentRatio"].fillna(quarter_file["currentRatio"], inplace=True)
        quarter_file["OperatingMargin"] = quarter_file.apply(lambda row:row["OperatingIncome"]/row["Revenue"] if row["Revenue"] >0 else None, axis=1)
        quarter_file["NetProfitMargin"] = quarter_file.apply(lambda row:row["NetIncome"]/row["Revenue"] if row["Revenue"] >0 else None, axis=1)
        quarter_file["DebtToEquity"] = quarter_file.apply(lambda row:row["TotalLiability"]/row["CommonStockEquity"] if row["CommonStockEquity"] >0 else None, axis=1)
        quarter_file["DebtToEquity"].fillna(quarter_file["debtToEquity"], inplace=True)
        quarter_file["AssetsTurnover"] = quarter_file.apply(lambda row:row["Revenue"]/row["TotalAssets"] if row["TotalAssets"] >0 else None, axis=1)
        quarter_file["AssetsTurnover"].fillna(quarter_file["assetTurnover"], inplace=True)
        quarter_file.fillna(0,inplace=True)
        for i in ['roic','quickRatio','ReturnOnAssets', 'ReturnOnEquity', 'CurrentRatio', 'OperatingMargin','NetProfitMargin', 'DebtToEquity',]:
            quarter_file[i+"Increase"] = quarter_file[i].diff().fillna(method="bfill")
        for i in ['NetIncome', 'Revenue', 'OperatingIncome', 'GrossProfit']:
            quarter_file[i+"Growth"] = quarter_file[i].shift()
            quarter_file[i+"Growth"] = quarter_file.apply(lambda row:(row[i] - row[i+"Growth"])/row[i+"Growth"] if row[i+"Growth"] >0 else None, axis=1)
        quarter_file["NetIncomeGrowth"].fillna(quarter_file["netIncomeGrowth"], inplace=True)
        quarter_file["OperatingIncomeGrowth"].fillna(quarter_file["operatingIncomeGrowth"], inplace=True)
        quarter_file["GrossProfitGrowth"].fillna(quarter_file["grossProfitGrowth"], inplace=True)

        quarter_file.drop(['NetIncome',
        'Revenue', 'OperatingIncome', 'EPSDiluted', 'GrossProfit',
        'GrossProfitRatio', 'TotalAssets', 'CommonStockEquity', 'CurrentAssets',
        'CurrentLiability', 'TotalLiability', 'Inventory', 'RetainedEarnings',"assetTurnover","debtToEquity","currentRatio","roe","returnOnAssets",'shareholdersEquityPerShare','threeYNetIncomeGrowthPerShare', 'threeYRevenueGrowthPerShare',
        'threeYShareholdersEquityGrowthPerShare','fiveYNetIncomeGrowthPerShare', 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare',"netIncomeGrowth","grossProfitGrowth","operatingIncomeGrowth"], axis=1, inplace=True)
        quarter_file_numeric = quarter_file.select_dtypes(include=[np.number])
        quarter_file[quarter_file_numeric.columns] = quarter_file_numeric.fillna(quarter_file_numeric.mean())
        
        new_path = data_path/"fundamental data"/"company ratios"/symbol
        if not new_path.exists():
            new_path.mkdir(parents=True)
        file_quarter = symbol+"_ratios_Quarter.csv"
        quarter_file.to_csv(new_path/file_quarter,index=False)

ttm_path = data_path/"fundamental data"/"year ratios"/"TTM"
if not ttm_path.exists():
    ttm_path.mkdir(parents=True)
file_ttm = "TTM_ratios_Year.csv"
ttm_data = ttm_data.set_index("Symbol")
ttm_data.to_csv(ttm_path/file_ttm)

In [85]:
for year in range(2009,2024):
    year_data = pd.DataFrame()
    year_str = str(year)
    new_path = data_path/"fundamental data"/"year ratios"/year_str
    if not new_path.exists():
        new_path.mkdir(parents=True)
    for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company ratios')]:
        path = data_path /"fundamental data"/ 'company ratios'/ symbol
        file = symbol+"_ratios_Year.csv"
        file_path = data_path /"fundamental data"/ 'company ratios' / symbol/file
        if file_path.exists():
            comp_data = pd.read_csv(file_path, index_col="Year")
            if int(year_str) in comp_data.index:
                year_data = pd.concat([year_data, comp_data.loc[[int(year_str)]].reset_index()], ignore_index=True)
    year_data = year_data.set_index("Symbol")
    year_file = year_str+"_ratios_Year.csv"
    year_data.to_csv(new_path/year_file)

In [92]:
for year in range(2009,2024):
    year_str = str(year)
    new_path = data_path/"fundamental data"/"year ratios"/year_str
    if not new_path.exists():
        new_path.mkdir(parents=True)
    for q in ["Q1", "Q2","Q3","Q4"]:
        q_data = pd.DataFrame()
        for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company ratios')]:
            path = data_path/"fundamental data" / 'company ratios'/ symbol
            file = symbol+"_ratios_Quarter.csv"
            file_path = data_path /"fundamental data"/ 'company ratios' / symbol/file
            if file_path.exists():
                comp_data = pd.read_csv(file_path, index_col=["Year", "Quarter"])
                if (year,q) in comp_data.index:
                    q_data = pd.concat([q_data, comp_data.loc[[(year,q)]].reset_index()], ignore_index=True)
        if not q_data.empty:
            q_data.Year = q_data.Year.astype(int).astype(str)
            q_data = q_data.set_index("Symbol")
            q_file = year_str+q+"_ratios_Q.csv"
            q_data.to_csv(new_path/q_file)